# Testzone
Script tester zone. Used to try code and ideas.

In [1]:
import utils
import os
import time

# machine learning
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE

In [ ]:
# program parameters
from dataclasses import dataclass

@dataclass
class ProgramParameters:
    """
    Wrapper class for program parameters.
    """
    MODEL_DIR_PATH = os.environ['HOME'] + "/Documents/code/phdtrack/Smart-and-Naive-SSH-Key-Extraction/smart_ssh_key_extractor/new_models"
    DATASET_DIR_PATH = os.environ['HOME'] + '/Documents/code/phdtrack/phdtrack_data/Performance_Test/Performance_Test/V_7_1_P1/'

    def __init__(self, **kwargs):
        if (
            utils.check_path_exists(self.MODEL_DIR_PATH) and
            utils.check_path_exists(self.DATASET_DIR_PATH)
        ):
            print("Program paths are OK.")
        else:
            print("Program paths are NOT OK.")
            exit(1)
        
        self.print_params()


PARAMS = ProgramParameters()

In [ ]:
# train a new classifier on the data

# Use SMOTE oversampling
start = time.time()
sm = SMOTE()
x_train, y_train = sm.fit_resample(dataset, labels)
end = time.time()
print('Time taken for resampling: %f' % (end - start))

train_data = utils.load_data(dataset, labels)
